In [1]:
import json
from tqdm import tqdm_notebook
import gzip
import pandas as pd
import sys
import os
import numpy as np
import json
from typing import Dict, List

In [2]:
from os import walk

f_zips = []
for (dirpath, dirnames, filenames) in walk('../../gorc/'):
    f_zips.extend(filenames)
    break

In [3]:
len(f_zips),f_zips

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

In [4]:
[file for file in f_zips if '.gz' not in file]

['s2orc-master.zip']

#### Выделение ACL статей,у которых есть  их статьи из reference c метаданными

##### Берем ACL статьи,у которых не менее 90% статьи из reference есть в датасете S2ORC

In [5]:
dir_name = 'acl_scenario/'

In [6]:
with open(dir_name+'acl_only_articles_0_9.json') as data_file:
    acl_only_articles = json.load(data_file)
data_file.close()

In [7]:
len(acl_only_articles)

7259

In [8]:
acl_only_articles_ids = list(map(lambda x: x['paper_id'],acl_only_articles))
len(acl_only_articles_ids)

7259

##### Берем данные статьи с overview

In [9]:
with open(dir_name+'overview_papers_0_9.json') as data_file:
    overview_papers = json.load(data_file)
data_file.close()

In [10]:
len(overview_papers)

7259

In [11]:
overview_papers_ids = list(map(lambda x: x['paper_id'],overview_papers))

In [12]:
len(set(overview_papers_ids)),len(set(acl_only_articles_ids)),len(set(overview_papers_ids+acl_only_articles_ids))

(7259, 7259, 7259)

**все верно**

##### Открываем все ACL OUT papers с метаданными и аннотациями

In [13]:
files=os.listdir(dir_name)

In [14]:
for file in tqdm_notebook(files):
    if ('acl_out_new_' in file) and ('json' in file) and not ('_0_8_'  in file):
        file = dir_name+file
        print(file)

acl_scenario/acl_out_new_0.json
acl_scenario/acl_out_new_10000.json
acl_scenario/acl_out_new_2000.json
acl_scenario/acl_out_new_4000.json
acl_scenario/acl_out_new_6000.json
acl_scenario/acl_out_new_8000.json



In [15]:
all_out_articles = []

for file in tqdm_notebook(files):
    if ('acl_out_new_' in file) and ('json' in file) and not ('_0_8_'  in file):
        file = dir_name+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        print(len(context_dict_batch))
        all_out_articles+=context_dict_batch
        del context_dict_batch

acl_scenario/acl_out_new_0.json
5
acl_scenario/acl_out_new_10000.json
5419
acl_scenario/acl_out_new_2000.json
5327
acl_scenario/acl_out_new_4000.json
5290
acl_scenario/acl_out_new_6000.json
5441
acl_scenario/acl_out_new_8000.json
5405



In [16]:
len(all_out_articles)

26887

In [17]:
all_out_articles_ids = list(map(lambda x: x['paper_id'],all_out_articles))
len(all_out_articles_ids),len(set(all_out_articles_ids))

(26887, 26886)

In [18]:
all_out_articles_ids_dict = {k:1 for k in all_out_articles_ids}

In [19]:
with open('acl_scenario/acl_out_ids.txt', encoding="utf8") as fin:
    content = fin.readlines()
fin.close()

In [20]:
acl_out_ids = [x.strip() for x in content]

In [21]:
len(acl_out_ids),len(set(acl_out_ids))

(26887, 26887)

In [22]:
len([ids for ids in set(acl_out_ids) if ids not in all_out_articles_ids_dict])

1

**как видим все супер** 

в нашей подборке out статей, нет только 1

## Анализ подборки

In [23]:
acl_paper_ids = [article['metadata']['acl_id'] for article in acl_only_articles if article['metadata']['acl_id']]
len(acl_paper_ids)

7259

##### Удаление дублирования по acl_id статей

In [26]:
doubled_acl_id_papers = []
for ind,cnt_of_acl_id in zip(pd.Series(acl_paper_ids).value_counts().index,pd.Series(acl_paper_ids).value_counts()):
    if cnt_of_acl_id >=2:
        doubled_acl_id_papers.append(ind)

In [27]:
del_items = []
del_num_items = []
for num_artic, article in enumerate(acl_only_articles):
    if article['metadata']['acl_id'] in doubled_acl_id_papers:
        del_items.append(article['paper_id'])
        del_num_items.append(num_artic)

In [28]:
len(del_items)

0

In [29]:
acl_paper_ids = [article['metadata']['acl_id'] for article in acl_only_articles if article['metadata']['acl_id']]
len(acl_paper_ids)

7259

In [30]:
def delete_items_from_papers(acl_ids_not_body_text,acl_only_articles):
    del_items = []
    del_num_items = []
    for num_artic, article in enumerate(acl_only_articles):
        if article['paper_id'] in acl_ids_not_body_text:
            del_items.append(article['paper_id'])
            del_num_items.append(num_artic)
            
    del_num_items = np.array(del_num_items)
    acl_only_articles = np.array(acl_only_articles)
    acl_only_articles = np.delete(acl_only_articles,del_num_items)
    return acl_only_articles

##### количество  всех статьей c body_text 

для acl_only_articles

In [31]:
len([article['paper_id'] for article in acl_only_articles if article['grobid_parse']['body_text'] or (article['latex_parse'] and article['latex_parse']['body_text'])])

7259

In [32]:
all_out_articles[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse_abstract', 'latex_parse_abstract', 'grobid_parse_bib_entries', 'latex_parse_bib_entries'])

In [33]:
all_out_articles[0]

{'paper_id': '13756507',
 'metadata': {'title': 'Collaborative Memory Network for Recommendation Systems',
  'authors': [{'first': 'Travis', 'middle': [], 'last': 'Ebesu', 'suffix': ''},
   {'first': 'Bin', 'middle': [], 'last': 'Shen', 'suffix': ''},
   {'first': 'Yi', 'middle': [], 'last': 'Fang', 'suffix': ''}],
  'abstract': "Recommendation systems play a vital role to keep users engaged with personalized content in modern online platforms. Deep learning has revolutionized many research fields and there is a recent surge of interest in applying it to collaborative filtering (CF). However, existing methods compose deep learning architectures with the latent factor model ignoring a major class of CF models, neighborhood or memory-based approaches. We propose Collaborative Memory Networks (CMN), a deep architecture to unify the two classes of CF models capitalizing on the strengths of the global structure of latent factor model and local neighborhood-based structure in a nonlinear fas

In [34]:
len([article['paper_id'] for article in all_out_articles if article['grobid_parse_abstract'] or article['latex_parse_abstract']])

15568

Проверка ситуации, когда есть **latex_parse**, но  нет **grobid_parse**

In [35]:
for num_art ,article in enumerate(acl_only_articles):
    if article['latex_parse'] and article['latex_parse']['body_text']:
        if len(article['grobid_parse']['body_text'])==0:
            print(num_art,article['paper_id'])

In [36]:
len([1 for num_art ,article in enumerate(all_out_articles) if article['latex_parse_abstract'] and not article['grobid_parse_abstract']])

0

## Формирование признакового пространства по каждой статье

признаки по каждой статье:
- цитируемость автора и статьи
- дата выхода статьи
- тематические признаки:
   - тема ref статьи общая с темой
     исходной статьи
   - тема ref статьи общая с темой
     с темой подбор
- мб журнал где была ссылка

In [37]:
all_out_articles[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse_abstract', 'latex_parse_abstract', 'grobid_parse_bib_entries', 'latex_parse_bib_entries'])

In [38]:
all_out_articles[0]['grobid_parse_abstract'][0]['text']

"ABSTRACTRecommendation systems play a vital role to keep users engaged with personalized content in modern online platforms. Deep learning has revolutionized many research fields and there is a recent surge of interest in applying it to collaborative filtering (CF). However, existing methods compose deep learning architectures with the latent factor model ignoring a major class of CF models, neighborhood or memory-based approaches. We propose Collaborative Memory Networks (CMN), a deep architecture to unify the two classes of CF models capitalizing on the strengths of the global structure of latent factor model and local neighborhood-based structure in a nonlinear fashion. Motivated by the success of Memory Networks, we fuse a memory component and neural attention mechanism as the neighborhood component. The associative addressing scheme with the user and item memories in the memory module encodes complex user-item relations coupled with the neural attention mechanism to learn a user-

In [39]:
print(all_out_articles[3]['paper_id'])
print(all_out_articles[3]['s2_pdf_hash'])
all_out_articles[0]['grobid_parse_abstract']

2497200
5e3cd43b9c0288a08c782b8df1311dd3e4f06c9b


[{'text': "ABSTRACTRecommendation systems play a vital role to keep users engaged with personalized content in modern online platforms. Deep learning has revolutionized many research fields and there is a recent surge of interest in applying it to collaborative filtering (CF). However, existing methods compose deep learning architectures with the latent factor model ignoring a major class of CF models, neighborhood or memory-based approaches. We propose Collaborative Memory Networks (CMN), a deep architecture to unify the two classes of CF models capitalizing on the strengths of the global structure of latent factor model and local neighborhood-based structure in a nonlinear fashion. Motivated by the success of Memory Networks, we fuse a memory component and neural attention mechanism as the neighborhood component. The associative addressing scheme with the user and item memories in the memory module encodes complex user-item relations coupled with the neural attention mechanism to lea

In [40]:
p_ids = []
p_s2pdf = []
p_year = []
p_arxiv = []
p_acl = []
p_pmc = []
p_pubmed = []
p_doi = []
p_venue = []
p_journal = []
p_abstract = []
p_abstract_grobid = []
p_abstract_latex = []
p_title = []
p_authors = []
p_cnt_out_grobid = []
p_cnt_out_latex = []
for article in all_out_articles:
    p_ids.append(article['paper_id'])
    p_s2pdf.append(article['s2_pdf_hash'])
    
    if article['metadata']['year']:
        p_year.append(article['metadata']['year'])
    else:
        p_year.append(0)
        
    if article['metadata']['arxiv_id']:
        p_arxiv.append(1)
    else:
        p_arxiv.append(0)
    if article['metadata']['acl_id']:
        p_acl.append(1)
    else:
        p_acl.append(0)
    if article['metadata']['pmc_id']:
        p_pmc.append(1)
    else:
        p_pmc.append(0)
    if article['metadata']['pubmed_id']:
        p_pubmed.append(1)
    else:
        p_pubmed.append(0)
    if article['metadata']['doi']:
        p_doi.append(1)
    else:
        p_doi.append(0)
        
    if article['metadata']['venue']:
        p_venue.append(article['metadata']['venue'])
    else:
        p_venue.append(0)
    if article['metadata']['journal']:
        p_journal.append(article['metadata']['journal'])
    else:
        p_journal.append(0)
        
    if article['metadata']['abstract']:
        p_abstract.append(article['metadata']['abstract'])
    else:
        p_abstract.append(0)
    if article['metadata']['title']:
        p_title.append(article['metadata']['title'])
    else:
        p_title.append(0)
    if article['grobid_parse_abstract']:
        p_abstract_grobid.append(article['grobid_parse_abstract'][0]['text'])
    else:
        p_abstract_grobid.append(0)
    if article['latex_parse_abstract']:
        p_abstract_latex.append(article['latex_parse_abstract'][0]['text'])
    else:
        p_abstract_latex.append(0)
    
        
    if article['grobid_parse_bib_entries']:
        p_cnt_out_grobid.append(len(article['grobid_parse_bib_entries']))
    else:
        p_cnt_out_grobid.append(0)
    if  article['latex_parse_bib_entries']:
        p_cnt_out_latex.append(len(article['latex_parse_bib_entries']))
    else:
        p_cnt_out_latex.append(0)
    
    authors = []
    if article['metadata']['authors']:
        for author in article['metadata']['authors']:
            if author['first']:
                authors.append(author['first']+' '+author['last'])
            elif author['middle'] and author['suffix']:
                print('can be ERROR',article['paper_id'])
            elif author['last']:
                authors.append(author['last'])
            else:
                print('can be ERROR',article['paper_id'])
    p_authors.append(authors)

In [285]:
acl_papers_features = {'paper_id': p_ids,'s2_pdf_hash':p_s2pdf,'year':p_year,
                       'arxiv_id': p_arxiv,'acl_id':p_acl,'pmc_id':p_pmc,'pubmed_id':p_pubmed,'doi':p_doi,
                       'venue': p_venue,'journal': p_journal,
                       'abstract':p_abstract,'title':p_title,'authors':p_authors,
                       'abstract_grobid':p_abstract_grobid,'abstract_latex':p_abstract_latex,
                       'cnt_out_grobid':p_cnt_out_grobid,'cnt_out_latex':p_cnt_out_latex}

In [286]:
df_acl_features  = pd.DataFrame(acl_papers_features)
df_acl_features.index = p_ids

In [287]:
df_acl_features.head(20)

,paper_id,s2_pdf_hash,year,arxiv_id,acl_id,pmc_id,pubmed_id,doi,venue,journal,abstract,title,authors,abstract_grobid,abstract_latex,cnt_out_grobid,cnt_out_latex
13756507,13756507,3944c1740bb643415583363a0a35f12f8bd19a16,2018,1,0,0,0,1,ArXiv,ArXiv,Recommendation systems play a vital role to ke...,Collaborative Memory Network for Recommendatio...,"[Travis Ebesu, Bin Shen, Yi Fang]",ABSTRACTRecommendation systems play a vital ro...,0,40,41
14472576,14472576,f3de408be7d2e2720a61451bd196ac7e1ed9363a,2015,0,1,0,0,1,ACL,Proceedings of the 53rd Annual Meeting of the ...,How do we build a semantic parser in a new dom...,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",AbstractHow do we build a semantic parser in a...,0,27,0
21369026,21369026,abd6baaf7e9d830feb3889e1019a989b6959ae89,2017,0,0,0,0,1,TPDL,Lecture Notes in Computer Science,0,On the Uses of Word Sense Change for Research ...,"[Nina Tahmasebi, Thomas Risse]",0,0,29,0
2497200,2497200,5e3cd43b9c0288a08c782b8df1311dd3e4f06c9b,1994,0,0,0,0,1,TOIS,ACM Trans. Inf. Syst.,We describe an approach to text classification...,Information extraction as a basis for high-pre...,"[Ellen Riloff, Wendy Lehnert]",0,0,0,0
406684,406684,a68dc3c376272c5c08df211c6974f02380295ebd,2005,0,0,0,0,1,IEEE Transactions on Knowledge and Data Engine...,IEEE Transactions on Knowledge and Data Engine...,"In many practical data mining applications, su...",Tri-training: exploiting unlabeled data using ...,"[Zhi-Hua Zhou, Ming Li]",0,0,30,0
4964135,4964135,e540e8c852ad8e7e80f2dda4c3deae536bd544aa,2009,0,0,0,0,1,ICML '09,Proceedings of the 26th Annual International C...,We propose a multiple source domain adaptation...,Domain adaptation from multiple sources via au...,"[Lixin Duan, Ivor Tsang, Dong Xu, Tat-Seng Chua]",0,0,20,0
6222370,6222370,08823db4f7acd0a72ffdbac4f1adfd3e52e22516,2006,0,0,0,0,1,"IEEE Transactions on Audio, Speech, and Langua...","IEEE Transactions on Audio, Speech, and Langua...",This paper describes a method for obtaining a ...,A chorus section detection method for musical ...,[M. Goto],0,0,30,0
34547425,34547425,1e21d0480f2c03859461dc7abd1c11b137bd5dcf,2017,1,0,0,0,0,ArXiv,ArXiv,"This paper introduces THUMT, an open-source to...",THUMT: An Open Source Toolkit for Neural Machi...,"[Jiacheng Zhang, Yanzhuo Ding, Shiqi Shen, Yon...","AbstractThis paper introduces THUMT, an openso...",0,17,15
53079238,53079238,792f5bf93d5e43b51a3f9ad4c1bb5af48bd69b2e,2018,0,1,0,0,1,EMNLP,Proceedings of the 2018 Conference on Empirica...,0,Improving Large-Scale Fact-Checking using Deco...,"[Nayeon Lee, Chien-Sheng Wu, Pascale Fung]",AbstractFact-checking of textual sources needs...,0,23,0
7431927,7431927,044490c14a54471055564043c19d23434b29da2d,2012,0,1,0,0,0,ACL,0,0,Crosslingual Induction of Semantic Roles,"[Ivan Titov, Alexandre Klementiev]",AbstractWe argue that multilingual parallel da...,0,52,0


In [288]:
events = {
    'ranlp':'Recent Advances in Natural Language Processing',
    'anlp':'Applied Natural Language Processing',
    'conll':'Computational Natural Language Learning',
    'eacl':'European Chapter of the Association for Computational Linguistic',
    'emnlp':'Empirical Methods in Natural Language Processing',
    'naacl':'North American Chapter of the Association for Computational Linguistic',
    'semeval':'Semantic Evaluation',#Lexical and Computational Semantics and 
    'tacl':'Transactions of the Association for Computational Linguistics',
    'alta':'Australasian Language Technology Association',
    'coling':'International Conference on Computational Linguistics',# aka ICCL
    'hlt':'Human Language Technology',
    'ijcnlp':'International Joint Conference on Natural Language Processing',
    'jep-taln-recital':'JEP TALN RECITAL',
    'lrec':'Language Resources and Evaluation',
    'muc':'Message Understanding Conf',
    'paclic':'Pacific Asia Conference on Language, Information and Computation',
    'rocling-ijclclp':'Rocling Computation Linguistics',
    'tinlap':'Theoretical Issues In Natural Language Processing',
    'tipster':'TIPSTER',
    'acl':'Association for Computational Linguistic',
    'acl1':'Association of Computational Linguistic',
    'cl':'Computational Linguistics Journal',
    'senseval':'SENSEVAL',
    'dialogue':'dialogue'
}

In [289]:
val = df_acl_features.venue.values[0]
val

'ArXiv'

In [290]:
text = val.replace(',',' ').split()
text

['ArXiv']

In [291]:
def mult_in(list_toks,line):
    flag = False
    line = line.lower()
    for tok in list_toks:
        if tok.lower() in line:
            flag = tok
            break
    return flag

In [292]:
events_keys = list(events.keys())+['arxiv']

In [293]:
events_values = {item.lower():key.lower() for key,item in events.items()}

In [294]:
events_values_keys = events_values.keys()
events_values_keys

dict_keys(['recent advances in natural language processing', 'applied natural language processing', 'computational natural language learning', 'european chapter of the association for computational linguistic', 'empirical methods in natural language processing', 'north american chapter of the association for computational linguistic', 'semantic evaluation', 'transactions of the association for computational linguistics', 'australasian language technology association', 'international conference on computational linguistics', 'human language technology', 'international joint conference on natural language processing', 'jep taln recital', 'language resources and evaluation', 'message understanding conf', 'pacific asia conference on language, information and computation', 'rocling computation linguistics', 'theoretical issues in natural language processing', 'tipster', 'association for computational linguistic', 'association of computational linguistic', 'computational linguistics journal'

In [295]:
import re
s = "This must not be deleted, but the number at the end yes 134411"
s = "ICPR'06"
s = re.sub("\d+", "", s)
s

"ICPR'"

In [296]:
additional_venue_keys = ['journal','conference','workshop']

In [297]:
cnt = 0
for ind,val in zip(df_acl_features.index,df_acl_features.venue):
    if cnt == 100:
        break
    cnt+=1
    flag_continue = True
    if val == 0:
        print(ind,'nothing')
        continue
    if mult_in(events_keys,val.lower()):
#         print(ind,val,'--- Yes|mult_in =',mult_in(events_keys,val.lower()))
        continue
    if mult_in(events_values_keys,val.lower()):
        print(ind,val,'--- Yes|mult_in =',events_values[mult_in(events_values_keys,val.lower())])
        continue
    text = val.replace(',',' ').replace('-',' ').replace('+',' ').replace('(',' ').replace(')',' ').split()
    for word in text:
        word_upp_cnt = sum(map(str.isupper, word))
        if not flag_continue:
            break
        if word_upp_cnt>1:
            print(ind,val,'--- Yes|word_upp =',word)
            flag_continue =False
            
    if mult_in(additional_venue_keys,val.lower()):
        val = val.replace(',',' ').replace('-',' ').replace('+',' ').replace('(',' ').replace(')',' ')
        val = re.sub("\d+", "", val)
        print(ind,'<additional>',val.lower())
        flag_continue =False
    if flag_continue:
        print(ind,'other',val)

21369026 TPDL --- Yes|word_upp = TPDL
2497200 TOIS --- Yes|word_upp = TOIS
406684 IEEE Transactions on Knowledge and Data Engineering --- Yes|word_upp = IEEE
4964135 ICML '09 --- Yes|word_upp = ICML
6222370 IEEE Transactions on Audio, Speech, and Language Processing --- Yes|word_upp = IEEE
538820 <additional> journal of machine learning research
140282220 nothing
417913 Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies --- Yes|mult_in = acl
267094 Proceedings of the Third Workshop on Innovative Use of NLP for Building Educational Applications --- Yes|word_upp = NLP
267094 <additional> proceedings of the third workshop on innovative use of nlp for building educational applications
12547848 International Workshop On Paraphrasing IWP --- Yes|word_upp = IWP
12547848 <additional> international workshop on paraphrasing iwp
37306935 TAC --- Yes|word_upp = TAC
60510120 nothing
13748870 NeurIPS 2018 Proceedings --- Yes|word_upp 

In [298]:
def venue2acl_events(x):
    if x == 0:
        return 'nothing'
    if mult_in(events_keys,x.lower()):
        return mult_in(events_keys,x.lower())
    if mult_in(events_values_keys,x.lower()):
        return events_values[mult_in(events_values_keys,x.lower())]
    x = x.replace(',',' ').replace('-',' ').replace('+',' ').replace('(',' ').replace(')',' ').replace("’",' ').replace(":",' ').replace("'",' ').replace("/",' ')
    text = re.sub("\d+", "", x)
    text = text.split()
    for word in text:
        word_upp_cnt = sum(map(str.isupper, word))
        if word_upp_cnt>1:
            return word        
    if mult_in(additional_venue_keys,x.lower()):
        return x.lower()
    return 'other'

In [299]:
k = 10
df_acl_features.venue[k],venue2acl_events(df_acl_features.venue[k])

('ACL', 'acl')

In [300]:
df_acl_features['new_venue'] = df_acl_features.venue.apply(venue2acl_events)

In [301]:
new_venue_val_counts  = {ind:val for ind,val in zip(df_acl_features.new_venue.value_counts().index,df_acl_features.new_venue.value_counts())}
new_venue_val_counts

{'arxiv': 3450,
 'other': 2679,
 'acl': 2257,
 'nothing': 1788,
 'emnlp': 1602,
 'naacl': 1315,
 'coling': 1163,
 'IEEE': 1144,
 'lrec': 578,
 'conll': 435,
 'cl': 422,
 'eacl': 406,
 'AAAI': 406,
 'ijcnlp': 402,
 'SIGIR': 378,
 'CIKM': 287,
 'IJCAI': 254,
 'WWW': 247,
 'INTERSPEECH': 234,
 'ICML': 229,
 'NIPS': 211,
 'KDD': 162,
 'semeval': 153,
 'SIGDIAL': 136,
 'tacl': 124,
 'ICWSM': 103,
 'hlt': 98,
 'acl1': 96,
 'BMC': 96,
 'WSDM': 93,
 'JAMIA': 92,
 'WMT': 86,
 'ranlp': 84,
 'BioNLP': 74,
 'NLP': 72,
 'CHI': 70,
 'anlp': 67,
 'AMIA': 65,
 'IN': 64,
 'TAC': 61,
 'journal of biomedical informatics': 59,
 'ENLG': 57,
 'ECIR': 54,
 'INLG': 49,
 'dialogue': 47,
 'ICSLP': 47,
 'IWSLT': 46,
 'paclic': 43,
 'IWCS': 42,
 'ACM': 42,
 'CACM': 41,
 'TREC': 41,
 'alta': 40,
 'workshop on statistical machine translation': 38,
 'PloS': 38,
 'IUI': 37,
 'NODALIDA': 34,
 'ICMI': 33,
 'ECML': 33,
 'FLAIRS': 32,
 'journal of machine learning research': 32,
 'AMTA': 32,
 'proceedings of the sixth wo

In [302]:
len(new_venue_val_counts)

1659

In [303]:
# len(new_venue_val_counts) old = 1657

In [304]:
# оставим только конференции, которые были выше какого-то treshold hfp
def venue_limitation(x,threshold=14):
    if new_venue_val_counts[x]>=threshold:
        return x
    else:
        return 'other'

In [305]:
df_acl_features.venue[k]

'ACL'

In [306]:
df_acl_features.venue = df_acl_features.new_venue.apply(venue_limitation)
df_acl_features.drop('new_venue',axis=1,inplace=True)

In [307]:
df_acl_features.drop('journal',axis=1,inplace=True)

In [308]:
# оставим только конференции, которые были выше какого-то treshold hfp
def venue_citation(x,threshold=14):
    if x == 'other' or x == 'nothing':
        return 0
    return new_venue_val_counts[x]

In [309]:
df_acl_features['venue_cit_n'] = df_acl_features.venue.apply(venue_citation)

In [310]:
df_acl_features.head()

,paper_id,s2_pdf_hash,year,arxiv_id,acl_id,pmc_id,pubmed_id,doi,venue,abstract,title,authors,abstract_grobid,abstract_latex,cnt_out_grobid,cnt_out_latex,venue_cit_n
13756507,13756507,3944c1740bb643415583363a0a35f12f8bd19a16,2018,1,0,0,0,1,arxiv,Recommendation systems play a vital role to ke...,Collaborative Memory Network for Recommendatio...,"[Travis Ebesu, Bin Shen, Yi Fang]",ABSTRACTRecommendation systems play a vital ro...,0,40,41,3450
14472576,14472576,f3de408be7d2e2720a61451bd196ac7e1ed9363a,2015,0,1,0,0,1,acl,How do we build a semantic parser in a new dom...,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",AbstractHow do we build a semantic parser in a...,0,27,0,2257
21369026,21369026,abd6baaf7e9d830feb3889e1019a989b6959ae89,2017,0,0,0,0,1,other,0,On the Uses of Word Sense Change for Research ...,"[Nina Tahmasebi, Thomas Risse]",0,0,29,0,0
2497200,2497200,5e3cd43b9c0288a08c782b8df1311dd3e4f06c9b,1994,0,0,0,0,1,TOIS,We describe an approach to text classification...,Information extraction as a basis for high-pre...,"[Ellen Riloff, Wendy Lehnert]",0,0,0,0,30
406684,406684,a68dc3c376272c5c08df211c6974f02380295ebd,2005,0,0,0,0,1,IEEE,"In many practical data mining applications, su...",Tri-training: exploiting unlabeled data using ...,"[Zhi-Hua Zhou, Ming Li]",0,0,30,0,1144


##### признак цитируемость статьи

In [312]:
with open('dict_in/dict_in_new.json') as data_file:
    dict_in = json.load(data_file)
data_file.close()

In [313]:
list(dict_in.keys())[:10]

['73647128',
 '3521928',
 '52799282',
 '13808052',
 '26463',
 '26599055',
 '4697457',
 '18665195',
 '11212020',
 '8180128']

In [314]:
p_in_cit = []
dict_in_keys = {k:1 for k in dict_in.keys()}
for num_artic,article in enumerate(all_out_articles):
#     if num_artic > 20:
#         break
    if article['paper_id'] in dict_in_keys:
        p_in_cit.append(len(dict_in[article['paper_id']]['in']))
    else:
        p_in_cit.append(0)

In [315]:
df_acl_features['in_cits'] = p_in_cit

In [316]:
df_acl_features.tail(10)

,paper_id,s2_pdf_hash,year,arxiv_id,acl_id,pmc_id,pubmed_id,doi,venue,abstract,title,authors,abstract_grobid,abstract_latex,cnt_out_grobid,cnt_out_latex,venue_cit_n,in_cits
16579379,16579379,8a7c82d6efb26ad725a80ecdf97acecd09f0adab,2016,0,1,0,0,1,naacl,0,MayoNLP at SemEval-2016 Task 1: Semantic Textu...,"[Naveed Afzal, Yanshan Wang, Hongfang Liu]","AbstractGiven two sentences, participating sys...",0,20,0,1315,4
7565460,7565460,f0ec25cb2e7a17fed5eae185a5b779c1c0704719,2015,0,1,0,0,1,acl,Community question answering (cQA) has become ...,Learning Continuous Word Embedding with Metada...,"[Guangyou Zhou, Tingting He, Jun Zhao, Po Hu]",AbstractCommunity question answering (cQA) has...,0,30,0,2257,35
2990641,2990641,9207dda4344cd2e07319ec1fdba3e1124dd9b09b,2009,0,0,0,0,0,ICWSM,0,Connecting Corresponding Identities across Com...,"[Reza Zafarani, Huan Liu]",0,0,1,0,103,0
10103917,10103917,2d31c7839cdf0ba8f778688e7a2b0d513cce3e4c,2011,0,1,0,0,0,conll,0,Mention Detection: Heuristics for the OntoNote...,"[Jonathan Kummerfeld, Mohit Bansal, David Burk...",AbstractOur submission was a reduced version o...,0,4,0,435,5
2076914,2076914,378d8752535f9361db73c713eda2b91a05a8b42b,2009,0,1,0,0,1,emnlp,One of the most desired information types when...,Geo-mining: Discovery of Road and Transport Ne...,"[Dmitry Davidov, Ari Rappoport]",AbstractOne of the most desired information ty...,0,24,0,1602,4
3551784,3551784,f91d764f59a922e3d25ada4dd1b5f1dea7320662,2013,0,1,0,0,0,acl,0,Linking and Extending an Open Multilingual Wor...,"[Francis Bond, Ryan Foster]",AbstractWe create an open multilingual wordnet...,0,42,0,2257,59
793980,793980,0a995afa8d3c114b2b431c4e2737777a0e051bff,1992,0,0,0,1,1,other,"Accumulating neuropsychological, electrophysio...",Separate visual pathways for perception and ac...,"[Melvyn Goodale, A. Milner]",0,0,0,0,0,0
14273237,14273237,acb68e759e50098642a347640e84f748ea56f10b,1999,0,0,0,0,0,other,"Abstract : Unlike read or laboratory speech, s...",Phonetic Consequences Of Speech Disfluency,[Elizabeth Shriberg],0,0,21,0,0,0
1557484,1557484,f5fb3dd7d59b06c1e2a2c32fbd58598eab82e8f2,0,0,1,0,0,0,nothing,0,Efficient induction of probabilistic word clas...,[Grzegorz Chrupała],AbstractWord classes automatically induced fro...,0,40,0,0,5
18889305,18889305,ca0eb4e9f08cf6fed7cb5212451d9bc4026a3efc,2015,0,1,0,0,1,ijcnlp,0,Chinese Grammatical Error Diagnosis System Bas...,"[Xiupeng Wu, Peijie Huang, Jundong Wang, Qingw...",AbstractThis paper describes our system in the...,0,15,0,402,1


In [317]:
del dict_in
del dict_in_keys
del p_in_cit

### Заглушка для создания признака цитируемости автора 

## Выделение обзорной части статьи

1. Самый простой принцип построения - по максимальному количеству ссылок в абзаце:
 - **Решение**:
    - подсчитать количество ссылок в каждой секции
    - выбрать секцию с максимальным количеством ссылок (возможно ещё оставить ещё 1 секцию, в которой количество ссылок было больше половины чем в максимальной)
    - для latex статей надо объединить текст одинаковых секций в 1 абзац
 - **Критерий**:
    -  в части latex публикаций есть названия секций => после выделения обзорных часте можно посмотреть какие секции выделились: какие топ-3, сделать просмотр глазами и после этого решать что делать дальше.
    - возможно логично сохранять для 2 максимального текста название статей
    

In [76]:
grobid_parse_overview = dict()
for num_sec,sections in enumerate(acl_only_articles[0]['grobid_parse']['body_text']):
    grobid_parse_overview[num_sec] = sections
    print(len(grobid_parse_overview[num_sec]['cite_spans']),sections)
    print(10*'==')

13 {'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, wellorganized summary for a topic which describes an event with a set of documents from different sources. (Goldstein et al., 2000; Erkan and Radev, 2004; Wan et al., 2007; Nenkova and McKeown, 2012; Min et al., 2012; Li et al., 2017) . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.With the development of social media and mobile equipments, more and more user generated * The work described in this paper is supported by a grant from the Grant Council of the Hong Kong Special Administrative Region, China (Project Code: 14203414) .1 http://www.se.cuhk.edu.hk/˜textmine/ dataset/ra-mds/ NEWS: The most important announcements from Google\'s big developers\' conference content is available. Figure 1 is a snapshot of rea

0 {'text': 'In this section, we describe the preparation process of the dataset. Then we provide some properties and statistics.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': None}
0 {'text': 'The definition of the terminology related to the dataset is given as follows. 6 Topic: A topic refers to an event and it is composed of a set of news documents from different sources. Document: A news article describing some aspects of the topic. The set of documents in the same topic typically span a period, say a few days. Category: Each topic belongs to a category. There are 6 predefined categories: (1) Accidents and Natural Disasters, (2) Attacks (Criminal/Terrorist), (3) New Technology, (4) Health and Safety, (5) Endangered Resources, and (6) Investigations and Trials (Criminal/Legal/Other). Aspect: Each category has a set of predefined aspects. Each aspect describes one important element of an event. For example, for the category "Accidents and Natural Disasters", the aspe

In [77]:
grobid_parse_overview

{0: {'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, wellorganized summary for a topic which describes an event with a set of documents from different sources. (Goldstein et al., 2000; Erkan and Radev, 2004; Wan et al., 2007; Nenkova and McKeown, 2012; Min et al., 2012; Li et al., 2017) . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.With the development of social media and mobile equipments, more and more user generated * The work described in this paper is supported by a grant from the Grant Council of the Hong Kong Special Administrative Region, China (Project Code: 14203414) .1 http://www.se.cuhk.edu.hk/˜textmine/ dataset/ra-mds/ NEWS: The most important announcements from Google\'s big developers\' conference content is available. Figure 1 is a snapshot of re

In [78]:
latex_parse_overview = dict()
for sections in acl_only_articles[0]['latex_parse']['body_text']:
    latex_parse_overview[sections['section']] = sections
    print(sections['section'],len(sections['cite_spans']))
    print(sections)
    print(10*'==')

Introduction 7
{'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.', 'cite_spans': [{'start': 193, 'end': 200, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'}, {'start': 203, 'end': 210, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'}, {'start': 213, 'end': 220, 'text': None, 'latex': None, 'ref_id': 'BIBREF2'}, {'start': 223, 'end': 230, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'}, {'start': 233, 'end': 240, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'}, {'start': 243, 'end': 250, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'}, 

{'text': 'VAESum BIBREF6 employs an alignment mechanism BIBREF12 , BIBREF13 to recall the lost detailed information from the input sentence. Inspired this idea, we design a jointly weighted alignment mechanism by considering the news sentence and the comment sentence simultaneously. For each decoder hidden state INLINEFORM0 , we align it with each news encoder hidden state INLINEFORM1 by an alignment vector INLINEFORM2 . We also align it with each comments encoder hidden state INLINEFORM3 by an alignment vector INLINEFORM4 . In order to filter the noisy information from the comments, we again employ the comment weight INLINEFORM5 to adjust the alignment vector of comments: DISPLAYFORM0 ', 'cite_spans': [{'start': 7, 'end': 14, 'text': None, 'latex': None, 'ref_id': 'BIBREF6'}, {'start': 46, 'end': 54, 'text': None, 'latex': None, 'ref_id': 'BIBREF12'}, {'start': 57, 'end': 65, 'text': None, 'latex': None, 'ref_id': 'BIBREF13'}], 'ref_spans': [], 'eq_spans': [{'start': 304, 'end': 315, 

{'text': 'Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': 'Data Collection'}
Data Collection 0
{'text': 'After fini

In [79]:
for article in acl_only_articles[0]['latex_parse']['body_text']:
    print(article['section'],len(article['cite_spans']))
    print(article['text'])
    print(10*'==')

Introduction 7
The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.
Introduction 0
With the development of social media and mobile equipments, more and more user generated content is available. Figure FIGREF2 is a snapshot of reader comments under the news report “The most important announcements from Google's big developers' conference”. The content of the original news report talks about some new products based on AI techniques. The news report generally conveys an enthusiastic tone. However, while some readers share similar enthusiasms, some others expre

Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.
Data Collection 0
After finishing the summary writing procedure, we employed another expert for scrutinizing the summaries. Each su

In [80]:
acl_only_articles[0]['latex_parse']['body_text'][0].keys()

dict_keys(['text', 'cite_spans', 'ref_spans', 'eq_spans', 'section'])

In [81]:
latex_parse_overview = dict()
for sections in acl_only_articles[0]['latex_parse']['body_text']:
    if sections['section'] in latex_parse_overview:
        # если есть дублирование, такое бывает у первых часте
        if latex_parse_overview[sections['section']] == sections:
            continue
        else:
            latex_parse_overview[sections['section']]['text'].append(sections['text'])
            latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
            latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
            latex_parse_overview[sections['section']]['section'].append(sections['section'])
#             latex_parse_overview[sections['section']]['section'].append(sections['section'])
    else:
        latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                      'cite_span_lens':[len(sections['cite_spans'])],
                                                      'section':[sections['section']], 
                                                      'bib_entries':acl_only_articles[0]['latex_parse']}
        
    print(sections['section'],len(sections['cite_spans']))
    print(sections)
    print(10*'==')

Introduction 7
{'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.', 'cite_spans': [{'start': 193, 'end': 200, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'}, {'start': 203, 'end': 210, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'}, {'start': 213, 'end': 220, 'text': None, 'latex': None, 'ref_id': 'BIBREF2'}, {'start': 223, 'end': 230, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'}, {'start': 233, 'end': 240, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'}, {'start': 243, 'end': 250, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'}, 

{'text': 'VAESum BIBREF6 employs an alignment mechanism BIBREF12 , BIBREF13 to recall the lost detailed information from the input sentence. Inspired this idea, we design a jointly weighted alignment mechanism by considering the news sentence and the comment sentence simultaneously. For each decoder hidden state INLINEFORM0 , we align it with each news encoder hidden state INLINEFORM1 by an alignment vector INLINEFORM2 . We also align it with each comments encoder hidden state INLINEFORM3 by an alignment vector INLINEFORM4 . In order to filter the noisy information from the comments, we again employ the comment weight INLINEFORM5 to adjust the alignment vector of comments: DISPLAYFORM0 ', 'cite_spans': [{'start': 7, 'end': 14, 'text': None, 'latex': None, 'ref_id': 'BIBREF6'}, {'start': 46, 'end': 54, 'text': None, 'latex': None, 'ref_id': 'BIBREF12'}, {'start': 57, 'end': 65, 'text': None, 'latex': None, 'ref_id': 'BIBREF13'}], 'ref_spans': [], 'eq_spans': [{'start': 304, 'end': 315, 

{'text': 'Each topic is assigned to 4 experts, who are major in journalism, to conduct the summary writing. The task of summary writing is divided into two phases, namely, aspect facet identification, and summary generation. For the aspect facet identification, the experts read and digested all the news documents and reader comments under the topic. Then for each aspect, the experts extracted the related facets from the news document. The summaries were generated based on the annotated aspect facets. When selecting facets, one consideration is those facets that are popular in both news documents and reader comments have higher priority. Next, the facets that are popular in news documents have the next priority. The generated summary should cover as many aspects as possible, and should be well-organized using complete sentences with a length restriction of 100 words.', 'cite_spans': [], 'ref_spans': [], 'eq_spans': [], 'section': 'Data Collection'}
Data Collection 0
{'text': 'After fini

In [82]:
latex_parse_overview['Introduction']

{'text': ['The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.',
  "With the development of social media and mobile equipments, more and more user generated content is available. Figure FIGREF2 is a snapshot of reader comments under the news report “The most important announcements from Google's big developers' conference”. The content of the original news report talks about some new products based on AI techniques. The news report generally conveys an enthusiastic tone. However, while some readers share similar enthusiasms, some others express their worri

In [83]:
acl_only_articles[0]['grobid_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

##### мы должны выделить только обзорную часть из текста, а все остальноё сохранить

In [84]:
overview_papers = dict()
overview_papers[acl_only_articles[0]['paper_id']] = {
    'paper_id':acl_only_articles[0]['paper_id'],   'metadata':acl_only_articles[0]['metadata'],
    's2_pdf_hash':acl_only_articles[0]['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}

In [85]:
overview_papers[acl_only_articles[0]['paper_id']]['grobid_parse'] = {'abstract':acl_only_articles[0]['grobid_parse']['abstract'],
                                                                'overview_text':grobid_parse_overview,  
                                                                'bib_entries':acl_only_articles[0]['grobid_parse']['bib_entries']}
overview_papers[acl_only_articles[0]['paper_id']]['latex_parse'] =  {'abstract':acl_only_articles[0]['latex_parse']['abstract'],
                                                                'overview_text':latex_parse_overview,  
                                                                'bib_entries':acl_only_articles[0]['latex_parse']['bib_entries']}

In [86]:
overview_papers

{'10164018': {'paper_id': '10164018',
  'metadata': {'title': 'Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset',
   'authors': [{'first': 'Piji', 'middle': [], 'last': 'Li', 'suffix': ''},
    {'first': 'Lidong', 'middle': [], 'last': 'Bing', 'suffix': ''},
    {'first': 'Wai', 'middle': [], 'last': 'Lam', 'suffix': ''}],
   'abstract': 'We investigate the problem of reader-aware multi-document summarization (RA-MDS) and introduce a new dataset for this problem. To tackle RA-MDS, we extend a variational auto-encodes (VAEs) based MDS framework by jointly considering news documents and reader comments. To conduct evaluation for summarization performance, we prepare a new dataset. We describe the methods for data collection, aspect annotation, and summary writing as well as scrutinizing by experts. Experimental results show that reader comments can improve the summarization performance, which also demonstrates the usefulness of the proposed dataset. The 

Для сравнения глазами ссылка на [статью](https://arxiv.org/pdf/1708.01065.pdf)

Все супер

In [87]:
for num_tex,(k,v) in enumerate(latex_parse_overview.items()):
    print(num_tex,k,sum(v['cite_span_lens']))

0 Introduction 13
1 Overview 1
2 Reader-Aware Salience Estimation 6
3 Summary Construction 6
4 Data Description 0
5 Background 0
6 Data Collection 0
7 Data Properties 0
8 Dataset and Metrics 1
9 Comparative Methods 6
10 Experimental Settings 2
11 Results on Our Dataset 0
12 Further Investigation of Our Framework  2
13 Case Study 0
14 Conclusions 0


In [88]:
for k,v in grobid_parse_overview.items():
    print(k,len(v['cite_spans']))

0 13
1 1
2 6
3 3
4 0
5 0
6 0
7 0
8 1
9 4
10 2
11 2
12 0
13 1


Упорядочим по количеству ссылок и возьмём абзац с максимальным значением, а также со 2 максимальным значением если количество ссылок в нём больше половины от максимального кол-ва 

In [89]:
grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), 
                                                 key=lambda item: len(item[1]['cite_spans']), reverse=True)}

In [90]:
max_cite_span_sum = 0
max_grobid_parse_overview = dict()
for k,v in grobid_parse_overview.items():
    if max_cite_span_sum < len(v['cite_spans']):
        max_cite_span_sum = len(v['cite_spans'])
        max_grobid_parse_overview[k] = v
    elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
        max_grobid_parse_overview[k] = v

In [91]:
latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), 
                                                 key=lambda item: sum(item[1]['cite_span_lens']), reverse=True)}

In [92]:
max_cite_span_sum = 0
max_latex_parse_overview = dict()
for k,v in latex_parse_overview.items():
    if max_cite_span_sum < sum(v['cite_span_lens']):
        max_cite_span_sum = sum(v['cite_span_lens'])
        max_latex_parse_overview[k] = v
    elif (max_cite_span_sum>7) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
        max_latex_parse_overview[k] = v

In [93]:
for k,v in latex_parse_overview.items():
    print(k,sum(v['cite_span_lens']))

Introduction 13
Reader-Aware Salience Estimation 6
Summary Construction 6
Comparative Methods 6
Experimental Settings 2
Further Investigation of Our Framework  2
Overview 1
Dataset and Metrics 1
Data Description 0
Background 0
Data Collection 0
Data Properties 0
Results on Our Dataset 0
Case Study 0
Conclusions 0


### Применим для всех 


#### Выделим обзорные части статей

In [94]:
overview_papers = dict()
for num_artic,article in enumerate(acl_only_articles):
    # проверяем что у статьи есть grobid_parse и latex_parse и естб текст
    if (article['grobid_parse'] and article['grobid_parse']['body_text']) or (article['latex_parse'] and article['latex_parse']['body_text']):
        # задаем шаблон отражения статьи в укороченном формате (чтобы занимать меньше памяти)
        overview_papers[article['paper_id']] = { 'paper_id':article['paper_id'],   'metadata':article['metadata'],
                                                 's2_pdf_hash':article['s2_pdf_hash'], 'grobid_parse':None,'latex_parse':None}
        
        grobid_parse_overview = None
        # если у статьи есть article['grobid_parse']['body_text']
        if article['grobid_parse'] and article['grobid_parse']['body_text']:
            grobid_parse_overview = dict()
            # проходим по каждому абзацу текста
            for num_sec,sections in enumerate(article['grobid_parse']['body_text']):
                grobid_parse_overview[num_sec] = sections
            
            # отсортируем по количеству цитат абзацы
            grobid_parse_overview = {k: v for k, v in sorted(grobid_parse_overview.items(), key=lambda item: len(item[1]['cite_spans']), reverse=True)}
            
#             # найдем 1 , пока без 2 максимума по обзорной части
            max_cite_span_sum = 0
            max_grobid_parse_overview = dict()
            for k,v in grobid_parse_overview.items():
                if max_cite_span_sum < len(v['cite_spans']):
                    max_cite_span_sum = len(v['cite_spans'])
                    max_grobid_parse_overview[k] = v
                    
#                 # записываем 2 максимум, если количество ссылок в егочасти больше половины от максимального 
#                 elif (max_cite_span_sum>7) and len(v['cite_spans'])>max_cite_span_sum//2:
#                     max_grobid_parse_overview[k] = v
                    
            
            grobid_parse_overview = max_grobid_parse_overview
            
        latex_parse_overview = None
        # если у статьи есть article['latex_parse']['body_text']
        if article['latex_parse'] and article['latex_parse']['body_text']:
            latex_parse_overview = dict()
            # проходим по каждому абзацу текста
            # в latex_parse 
            for sections in article['latex_parse']['body_text']:
                if sections['section'] in latex_parse_overview:
                    if latex_parse_overview[sections['section']] == sections:
                        continue
                    else:
                        latex_parse_overview[sections['section']]['text'].append(sections['text'])
                        latex_parse_overview[sections['section']]['cite_spans'].append(sections['cite_spans'])
                        latex_parse_overview[sections['section']]['cite_span_lens'].append(len(sections['cite_spans']))
                        latex_parse_overview[sections['section']]['section'].append(sections['section'])
                else:
                    latex_parse_overview[sections['section']] = {'text':[sections['text']],   'cite_spans':[sections['cite_spans']],
                                                                  'cite_span_lens':[len(sections['cite_spans'])],
                                                                  'section':[sections['section']]}
            latex_parse_overview = {k: v for k, v in sorted(latex_parse_overview.items(), key=lambda item: item[1]['cite_span_lens'], reverse=True)}
        
        
            max_cite_span_sum = 0
            max_latex_parse_overview = dict()
            for k,v in latex_parse_overview.items():
                if max_cite_span_sum < sum(v['cite_span_lens']):
                    max_cite_span_sum = sum(v['cite_span_lens'])
                    max_latex_parse_overview[k] = v
#                 elif (max_cite_span_sum>0) and sum(v['cite_span_lens'])>max_cite_span_sum//2:
#                     max_latex_parse_overview[k] = v
            
            latex_parse_overview = max_latex_parse_overview
        

        if grobid_parse_overview:
            overview_papers[article['paper_id']]['grobid_parse'] = {'abstract':None,
                                                        'overview_text':grobid_parse_overview,  
                                                        'bib_entries':None}
            if article['grobid_parse']['abstract']:
                overview_papers[article['paper_id']]['grobid_parse']['abstract'] = article['grobid_parse']['abstract']
            if article['grobid_parse']['bib_entries']:
                overview_papers[article['paper_id']]['grobid_parse']['bib_entries'] = article['grobid_parse']['bib_entries']
            
        if latex_parse_overview:            
            overview_papers[article['paper_id']]['latex_parse'] = {'abstract':None,
                                                                    'overview_text':latex_parse_overview,  
                                                                    'bib_entries':None}
            if article['latex_parse']['abstract']:
                overview_papers[article['paper_id']]['latex_parse']['abstract'] = article['latex_parse']['abstract']
            if article['latex_parse']['bib_entries']:
                overview_papers[article['paper_id']]['latex_parse']['bib_entries'] = article['latex_parse']['bib_entries']

In [95]:
overview_papers_list = np.array(list(overview_papers.values()))

In [96]:
len(overview_papers_list)

7259

In [97]:
dir_name = 'acl_scenario/'

In [98]:
n_samples = len(overview_papers_list)
# Shuffle at the start of epoch
indices = np.arange(n_samples)
batch_size = 5000
for start in range(0, n_samples, batch_size):
    name_json = dir_name+'acl_overview_'+str(start)+'.json' 
    print(name_json)
    end = min(start + batch_size, n_samples)
    batch_idx = indices[start:end]
    batch_overview = list(overview_papers_list[batch_idx])
    with open(name_json, 'w') as json_fil:
        json.dump(batch_overview, json_fil)
    json_fil.close()
    del batch_overview

acl_scenario/acl_overview_0.json
acl_scenario/acl_overview_5000.json


In [99]:
overview_papers['10164018']['grobid_parse']['overview_text'].keys()

dict_keys([0])

In [100]:
overview_papers['10164018']['latex_parse']['overview_text'].keys()

dict_keys(['Introduction'])

##### Выделим стать, у которых есть latex_parse

In [101]:
overview_papers_w_latex = {k:v for k,v in overview_papers.items() if v['latex_parse']}

In [102]:
len(overview_papers_w_latex)

1058

In [103]:
len([k for k,v in overview_papers.items() if v['grobid_parse']])

7259

In [104]:
def mult_in(list_toks,line):
    flag = False
    line = line.lower()
    for tok in list_toks:
        if tok.lower() in line:
            flag = tok
            break
    return flag

### Выделение порядка упоминания статей

In [105]:
list(overview_papers.keys())[:10]

['10164018',
 '14472576',
 '14323173',
 '442560',
 '44112954',
 '22103840',
 '15281193',
 '10335977',
 '5084110',
 '1368861']

In [106]:
print(overview_papers['10164018'].keys())
print(overview_papers['10164018']['grobid_parse'].keys())
print(overview_papers['10164018']['latex_parse'].keys())
print(overview_papers['10164018']['grobid_parse']['overview_text'][0].keys())
print(overview_papers['10164018']['latex_parse']['overview_text']['Introduction'].keys())

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])
dict_keys(['abstract', 'overview_text', 'bib_entries'])
dict_keys(['abstract', 'overview_text', 'bib_entries'])
dict_keys(['text', 'cite_spans', 'ref_spans', 'eq_spans', 'section'])
dict_keys(['text', 'cite_spans', 'cite_span_lens', 'section'])


In [107]:
key_id = '10164018'#'442560'

In [108]:
overview_papers[key_id]['grobid_parse']['overview_text']

{0: {'text': 'The goal of multi-document summarization (MDS) is to automatically generate a brief, wellorganized summary for a topic which describes an event with a set of documents from different sources. (Goldstein et al., 2000; Erkan and Radev, 2004; Wan et al., 2007; Nenkova and McKeown, 2012; Min et al., 2012; Li et al., 2017) . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.With the development of social media and mobile equipments, more and more user generated * The work described in this paper is supported by a grant from the Grant Council of the Hong Kong Special Administrative Region, China (Project Code: 14203414) .1 http://www.se.cuhk.edu.hk/˜textmine/ dataset/ra-mds/ NEWS: The most important announcements from Google\'s big developers\' conference content is available. Figure 1 is a snapshot of re

In [109]:
overview_papers[key_id]['grobid_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'b0',
  'title': 'Neural machine translation by jointly learning to align and translate',
  'authors': [{'first': 'Dzmitry',
    'middle': [],
    'last': 'Bahdanau',
    'suffix': ''},
   {'first': 'Kyunghyun', 'middle': [], 'last': 'Cho', 'suffix': ''},
   {'first': 'Yoshua', 'middle': [], 'last': 'Bengio', 'suffix': ''}],
  'year': 2015,
  'venue': 'ICLR',
  'volume': '',
  'issn': '',
  'pages': '',
  'other_ids': {},
  'links': '11212020'},
 'BIBREF1': {'ref_id': 'b1',
  'title': 'Theano: new features and speed improvements',
  'authors': [{'first': 'Frédéric',
    'middle': [],
    'last': 'Bastien',
    'suffix': ''},
   {'first': 'Pascal', 'middle': [], 'last': 'Lamblin', 'suffix': ''},
   {'first': 'Razvan', 'middle': [], 'last': 'Pascanu', 'suffix': ''},
   {'first': 'James', 'middle': [], 'last': 'Bergstra', 'suffix': ''},
   {'first': 'Ian', 'middle': [], 'last': 'Goodfellow', 'suffix': ''},
   {'first': 'Arnaud', 'middle': [], 'last': 'Bergeron', 'su

In [110]:
key_grobid = list(overview_papers[key_id]['grobid_parse']['overview_text'].keys())[0]
overview_papers[key_id]['grobid_parse']['overview_text'][key_grobid]['cite_spans']

[{'start': 192,
  'end': 216,
  'text': '(Goldstein et al., 2000;',
  'latex': None,
  'ref_id': 'BIBREF6'},
 {'start': 217,
  'end': 239,
  'text': 'Erkan and Radev, 2004;',
  'latex': None,
  'ref_id': 'BIBREF4'},
 {'start': 240,
  'end': 257,
  'text': 'Wan et al., 2007;',
  'latex': None,
  'ref_id': 'BIBREF19'},
 {'start': 258,
  'end': 284,
  'text': 'Nenkova and McKeown, 2012;',
  'latex': None,
  'ref_id': 'BIBREF16'},
 {'start': 285,
  'end': 302,
  'text': 'Min et al., 2012;',
  'latex': None,
  'ref_id': 'BIBREF15'},
 {'start': 303,
  'end': 319,
  'text': 'Li et al., 2017)',
  'latex': None,
  'ref_id': 'BIBREF11'},
 {'start': 773,
  'end': 797,
  'text': '(Project Code: 14203414)',
  'latex': None,
  'ref_id': None},
 {'start': 2288,
  'end': 2305,
  'text': '(Hu et al., 2008;',
  'latex': None,
  'ref_id': 'BIBREF7'},
 {'start': 2306,
  'end': 2324,
  'text': 'Yang et al., 2011)',
  'latex': None,
  'ref_id': 'BIBREF22'},
 {'start': 2582,
  'end': 2598,
  'text': 'Li et a

In [111]:
bib_entries_grobid = dict()
bib_refs = []
ref_order = []
paper = overview_papers[key_id]

key_grobid = list(paper['grobid_parse']['overview_text'].keys())[0]

for cite_span in paper['grobid_parse']['overview_text'][key_grobid]['cite_spans']:
    cite_ref = cite_span['ref_id']
    bib_refs.append(cite_ref)
    cited_paper_id = None
    if cite_ref in paper['grobid_parse']['bib_entries']:
        cited_paper_id = paper['grobid_parse']['bib_entries'][cite_ref]['links']
    if cited_paper_id:
        ref_order.append(cited_paper_id)
        print(cite_ref,cited_paper_id,paper['grobid_parse']['bib_entries'][cite_ref]['title'])
        bib_entries_grobid[paper['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
    elif cite_ref  in paper['grobid_parse']['bib_entries']:
        print(cite_ref,cited_paper_id,paper['grobid_parse']['bib_entries'][cite_ref]['title'])
        bib_entries_grobid[paper['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
    else:
        # неверно распознается, это редко статья
        print('kek',cite_ref,cited_paper_id)
pd.Series(bib_refs).value_counts()

BIBREF6 8294822 Multi-document summarization by sentence extraction
BIBREF4 10418456 Lexpagerank: Prestige in multi-document text summarization
BIBREF19 532313 Manifold-ranking based topic-focused multidocument summarization
BIBREF16 556431 A survey of text summarization techniques
BIBREF15 6317274 Exploiting category-specific information for multidocument summarization
BIBREF11 29562039 Salience estimation via variational auto-encoders for multi-document summarization
kek None None
BIBREF7 13723748 Comments-oriented document summarization: Understanding documents with readers' feedback
BIBREF22 704517 Social context summarization
BIBREF2 8377315 Abstractive multidocument summarization via phrase selection and merging
BIBREF11 29562039 Salience estimation via variational auto-encoders for multi-document summarization
BIBREF9 None Autoencoding variational bayes
BIBREF18 16895865 Stochastic backpropagation and approximate inference in deep generative models


BIBREF11    2
BIBREF16    1
BIBREF18    1
BIBREF2     1
BIBREF7     1
BIBREF9     1
BIBREF19    1
BIBREF15    1
BIBREF6     1
BIBREF22    1
BIBREF4     1
dtype: int64

In [112]:
ref_order

['8294822',
 '10418456',
 '532313',
 '556431',
 '6317274',
 '29562039',
 '13723748',
 '704517',
 '8377315',
 '29562039',
 '16895865']

In [113]:
bib_entries_grobid

{'Multi-document summarization by sentence extraction': '8294822',
 'Lexpagerank: Prestige in multi-document text summarization': '10418456',
 'Manifold-ranking based topic-focused multidocument summarization': '532313',
 'A survey of text summarization techniques': '556431',
 'Exploiting category-specific information for multidocument summarization': '6317274',
 'Salience estimation via variational auto-encoders for multi-document summarization': '29562039',
 "Comments-oriented document summarization: Understanding documents with readers' feedback": '13723748',
 'Social context summarization': '704517',
 'Abstractive multidocument summarization via phrase selection and merging': '8377315',
 'Autoencoding variational bayes': None,
 'Stochastic backpropagation and approximate inference in deep generative models': '16895865'}

In [114]:
overview_papers[key_id]['latex_parse']['overview_text']['Introduction']['text']

['The goal of multi-document summarization (MDS) is to automatically generate a brief, well-organized summary for a topic which describes an event with a set of documents from different sources. BIBREF0 , BIBREF1 , BIBREF2 , BIBREF3 , BIBREF4 , BIBREF5 , BIBREF6 . In the typical setting of MDS, the input is a set of news documents about the same topic. The output summary is a piece of short text document containing several sentences, generated only based on the input original documents.',
 "With the development of social media and mobile equipments, more and more user generated content is available. Figure FIGREF2 is a snapshot of reader comments under the news report “The most important announcements from Google's big developers' conference”. The content of the original news report talks about some new products based on AI techniques. The news report generally conveys an enthusiastic tone. However, while some readers share similar enthusiasms, some others express their worries about n

In [115]:
overview_papers[key_id]['latex_parse']['overview_text']['Introduction']['cite_spans']

[[{'start': 193, 'end': 200, 'text': None, 'latex': None, 'ref_id': 'BIBREF0'},
  {'start': 203, 'end': 210, 'text': None, 'latex': None, 'ref_id': 'BIBREF1'},
  {'start': 213, 'end': 220, 'text': None, 'latex': None, 'ref_id': 'BIBREF2'},
  {'start': 223, 'end': 230, 'text': None, 'latex': None, 'ref_id': 'BIBREF3'},
  {'start': 233, 'end': 240, 'text': None, 'latex': None, 'ref_id': 'BIBREF4'},
  {'start': 243, 'end': 250, 'text': None, 'latex': None, 'ref_id': 'BIBREF5'},
  {'start': 253,
   'end': 260,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF6'}],
 [],
 [{'start': 527, 'end': 534, 'text': None, 'latex': None, 'ref_id': 'BIBREF7'},
  {'start': 537, 'end': 544, 'text': None, 'latex': None, 'ref_id': 'BIBREF8'},
  {'start': 802,
   'end': 809,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF9'}],
 [{'start': 10, 'end': 17, 'text': None, 'latex': None, 'ref_id': 'BIBREF6'},
  {'start': 159,
   'end': 167,
   'text': None,
   'latex': None,
   'ref_id': 'BIBREF10'}

In [116]:
article = overview_papers[key_id]

bib_entries_latex = dict()
for cite_spans in article['latex_parse']['overview_text']['Introduction']['cite_spans']:
    if len(cite_spans)>1:
        for cite_span in cite_spans:
            cite_ref = cite_span['ref_id']
            cited_article_id = None
            if cite_ref in article['latex_parse']['bib_entries']:
                cited_article_id = article['latex_parse']['bib_entries'][cite_ref]['links']
                
            if cited_article_id:
                print(cite_ref,cited_article_id,article['latex_parse']['bib_entries'][cite_ref]['title'])
                bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
            elif cite_ref in article['latex_parse']['bib_entries']:
                print(cite_ref,cited_article_id)
                
                bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
            else:
                print('kek',cite_ref,cited_article_id)

BIBREF0 8294822 Multi-document summarization by sentence extraction
BIBREF1 10418456 Lexpagerank: Prestige in multi-document text summarization
kek BIBREF2 None
BIBREF3 556431 A survey of text summarization techniques
BIBREF4 6317274 Exploiting category-specific information for multi-document summarization
BIBREF5 8377315 Abstractive multi-document summarization via phrase selection and merging
BIBREF6 29562039 Salience estimation via variational auto-encoders for multi-document summarization
BIBREF7 13723748 Comments-oriented document summarization: Understanding documents with readers' feedback
BIBREF8 704517 Social context summarization
BIBREF9 14777460 Reader-aware multi-document summarization via sparse coding
BIBREF6 29562039 Salience estimation via variational auto-encoders for multi-document summarization
BIBREF10 15789289 Auto-encoding variational bayes
BIBREF11 16895865 Stochastic backpropagation and approximate inference in deep generative models


In [117]:
bib_entries_latex

{'Multi-document summarization by sentence extraction': '8294822',
 'Lexpagerank: Prestige in multi-document text summarization': '10418456',
 'A survey of text summarization techniques': '556431',
 'Exploiting category-specific information for multi-document summarization': '6317274',
 'Abstractive multi-document summarization via phrase selection and merging': '8377315',
 'Salience estimation via variational auto-encoders for multi-document summarization': '29562039',
 "Comments-oriented document summarization: Understanding documents with readers' feedback": '13723748',
 'Social context summarization': '704517',
 'Reader-aware multi-document summarization via sparse coding': '14777460',
 'Auto-encoding variational bayes': '15789289',
 'Stochastic backpropagation and approximate inference in deep generative models': '16895865'}

In [118]:
bib_entries_grobid

{'Multi-document summarization by sentence extraction': '8294822',
 'Lexpagerank: Prestige in multi-document text summarization': '10418456',
 'Manifold-ranking based topic-focused multidocument summarization': '532313',
 'A survey of text summarization techniques': '556431',
 'Exploiting category-specific information for multidocument summarization': '6317274',
 'Salience estimation via variational auto-encoders for multi-document summarization': '29562039',
 "Comments-oriented document summarization: Understanding documents with readers' feedback": '13723748',
 'Social context summarization': '704517',
 'Abstractive multidocument summarization via phrase selection and merging': '8377315',
 'Autoencoding variational bayes': None,
 'Stochastic backpropagation and approximate inference in deep generative models': '16895865'}

## Пока заглушка
Начинаем обход с bib_entries_grobid смотрим, что у нас всё одинаково матчится с bib_entries_latex
если есть различия то bib_entries_latex добавляем, пока не заматчится

In [119]:
# all_bib_entries = dict()
# if bib_entries_latex:
#     for k,v in bib_entries_latex.items():
#         all_bib_entries[k] = [v]
# if bib_entries_grobid:
#     for k,v in bib_entries_grobid.items():
#         if k in all_bib_entries:
#             if all_bib_entries[k][0] == v:
#                 continue
#             else:
#                 all_bib_entries[k].append(v)
#         else:
#             all_bib_entries[k] = [v]

#### Пока делаю только для статей с grobid_parse

тк статей с latex в 7 раз меньше

In [120]:
overview_papers['10164018'].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

In [121]:
overview_papers['10164018']['metadata'].keys()

dict_keys(['title', 'authors', 'abstract', 'year', 'arxiv_id', 'acl_id', 'pmc_id', 'pubmed_id', 'doi', 'venue', 'journal'])

In [122]:
overview_papers['10164018']['metadata']['title']

'Reader-Aware Multi-Document Summarization: An Enhanced Model and The First Dataset'

In [123]:
overview_papers['10164018']['metadata']['authors']

[{'first': 'Piji', 'middle': [], 'last': 'Li', 'suffix': ''},
 {'first': 'Lidong', 'middle': [], 'last': 'Bing', 'suffix': ''},
 {'first': 'Wai', 'middle': [], 'last': 'Lam', 'suffix': ''}]

In [124]:
for author in overview_papers['10164018']['metadata']['authors']:
    if author['first']:
        print(author['first']+' '+author['last'])
    else:
        print(author['last'])

Piji Li
Lidong Bing
Wai Lam


In [125]:
overview_papers['10164018']['grobid_parse']['abstract']

[{'text': 'AbstractWe investigate the problem of readeraware multi-document summarization (RA-MDS) and introduce a new dataset for this problem. To tackle RA-MDS, we extend a variational auto-encodes (VAEs) based MDS framework by jointly considering news documents and reader comments. To conduct evaluation for summarization performance, we prepare a new dataset. We describe the methods for data collection, aspect annotation, and summary writing as well as scrutinizing by experts. Experimental results show that reader comments can improve the summarization performance, which also demonstrates the usefulness of the proposed dataset. The annotated dataset for RA-MDS is available online 1 .',
  'cite_spans': [],
  'ref_spans': [],
  'eq_spans': [],
  'section': 'Abstract'}]

In [126]:
article_bibs = {}

p_ref_ind = []
p_ref_title = []
p_ref_abstract = []
p_ref_abstract_grobid = []
p_ref_abstract_latex = []
p_ref_order = []
p_ref_authors = []

for article in tqdm_notebook(overview_papers):
# for num_artic,article in enumerate(overview_papers):
#     if num_artic >10:
#         break
#     print(num_artic,article)
    article = overview_papers[article]
    # проверка наличия определленных значений статьи
    if not ((article['grobid_parse'] and article['grobid_parse']['bib_entries']) or (article['latex_parse'] and article['latex_parse']['bib_entries'])):
#         article_bibs[article['paper_id']] = 'None'
        continue
        
    bib_entries_grobid = None
    if article['grobid_parse'] and article['grobid_parse']['bib_entries'] and article['grobid_parse']['overview_text']:
        bib_entries_grobid = dict()
        ref_order = []
        key_grobid = list(article['grobid_parse']['overview_text'].keys())[0]
        for cite_span in article['grobid_parse']['overview_text'][key_grobid]['cite_spans']:
            cite_ref = cite_span['ref_id']
            cited_paper_id = None
            if cite_ref in article['grobid_parse']['bib_entries']:
                cited_paper_id = article['grobid_parse']['bib_entries'][cite_ref]['links']
            if cited_paper_id:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
                ref_order.append(cited_paper_id)
            elif cite_ref  in article['grobid_parse']['bib_entries']:
                bib_entries_grobid[article['grobid_parse']['bib_entries'][cite_ref]['title']] = cited_paper_id
        
        
        
#     bib_entries_latex = None
#     if article['latex_parse'] and article['latex_parse']['overview_text'] and article['latex_parse']['bib_entries']:
#         bib_entries_latex = dict()
        
#         key_tex = list(article['latex_parse']['overview_text'].keys())[0]
#         for cite_spans in article['latex_parse']['overview_text'][key_tex]['cite_spans']:
#             if len(cite_spans)>1:
#                 for cite_span in cite_spans:
#                     cite_ref = cite_span['ref_id']
#                     cited_article_id = None
#                     if cite_ref in article['latex_parse']['bib_entries']:
#                         cited_article_id = article['latex_parse']['bib_entries'][cite_ref]['links']
#                     if cited_article_id:
#                         bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
#                     elif cite_ref in article['latex_parse']['bib_entries']:
#                         bib_entries_latex[article['latex_parse']['bib_entries'][cite_ref]['title']] = cited_article_id
    
    
    all_bib_entries = dict()
#     if bib_entries_latex:
#         for k,v in bib_entries_latex.items():
#             if k == '':
#                 continue
                
#             if v:
#                 all_bib_entries[k] = [v]
#             else:
#                 all_bib_entries[k] = []

    if bib_entries_grobid:
        for k,v in bib_entries_grobid.items():
            if k == '':
                continue
                
            if k in all_bib_entries:
                if len(all_bib_entries[k]) == 0:
                    if v:
                        all_bib_entries[k] = [v]
                    else:
                        all_bib_entries[k] = []
                else:
                    if all_bib_entries[k][0] == v:
                        continue
                    else:
                        if v:
                            all_bib_entries[k].append(v)
                        else:
                            continue
            else:
                if v:
                    all_bib_entries[k] = [v]
                else:
                    all_bib_entries[k] = []
        article_bibs[article['paper_id']] = all_bib_entries
        
        p_ref_order.append(ref_order)
        p_ref_ind.append(article['paper_id'])
        
#         key_grobid = list(article['grobid_parse']['abstract'].keys())[0]
        if article['metadata']['abstract']:
            p_ref_abstract.append(article['metadata']['abstract'])
        else:
            p_ref_abstract.append(0)
        if article['metadata']['title']:
            p_ref_title.append(article['metadata']['title'])
        else:
            p_ref_title.append(0)
        if article['grobid_parse'] and article['grobid_parse']['abstract']:
            p_ref_abstract_grobid.append(article['grobid_parse']['abstract'][0]['text'])
        else:
            p_ref_abstract_grobid.append(0)
        if article['latex_parse'] and article['latex_parse']['abstract']:
            p_ref_abstract_latex.append(article['latex_parse']['abstract'][0]['text'])
        else:
            p_ref_abstract_latex.append(0)
        
        ref_author = []
        if article['metadata']['authors']:
            for author in article['metadata']['authors']:
                if author['first']:
                    ref_author.append(author['first']+' '+author['last'])
                elif author['suffix'] or author['middle']:
                    print('CAn be mistake',article['metadata']['authors'],article['paper_id'])
                    break
                else:
                    ref_author.append(author['last'])
        p_ref_authors.append(ref_author)

In [127]:
list(article_bibs.keys())[:5]

['10164018', '14472576', '14323173', '442560', '44112954']

In [128]:
article_bibs['10164018']

{'Multi-document summarization by sentence extraction': ['8294822'],
 'Lexpagerank: Prestige in multi-document text summarization': ['10418456'],
 'Manifold-ranking based topic-focused multidocument summarization': ['532313'],
 'A survey of text summarization techniques': ['556431'],
 'Exploiting category-specific information for multidocument summarization': ['6317274'],
 'Salience estimation via variational auto-encoders for multi-document summarization': ['29562039'],
 "Comments-oriented document summarization: Understanding documents with readers' feedback": ['13723748'],
 'Social context summarization': ['704517'],
 'Abstractive multidocument summarization via phrase selection and merging': ['8377315'],
 'Autoencoding variational bayes': [],
 'Stochastic backpropagation and approximate inference in deep generative models': ['16895865']}

In [129]:
p_ref_order[0]

['8294822',
 '10418456',
 '532313',
 '556431',
 '6317274',
 '29562039',
 '13723748',
 '704517',
 '8377315',
 '29562039',
 '16895865']

In [130]:
pd.Series(p_ref_order[0]).value_counts()

29562039    2
16895865    1
10418456    1
13723748    1
8294822     1
704517      1
6317274     1
556431      1
532313      1
8377315     1
dtype: int64

In [131]:
np.mean([len(p) for p in p_ref_order])

13.196445791431326

In [132]:
p_ref_ind[0]

'10164018'

In [218]:
target = {'acl_p_id': p_ref_ind,'title':p_ref_title,'authors':p_ref_authors,'abstract':p_ref_abstract,
          'abstract_grobid':p_ref_abstract_grobid,'abstract_latex':p_ref_abstract_latex,'ref_order': p_ref_order}

In [219]:
df_target = pd.DataFrame(target)
df_target.index = p_ref_ind

In [220]:
df_target.head()

,acl_p_id,title,authors,abstract,abstract_grobid,abstract_latex,ref_order
10164018,10164018,Reader-Aware Multi-Document Summarization: An ...,"[Piji Li, Lidong Bing, Wai Lam]",We investigate the problem of reader-aware mul...,AbstractWe investigate the problem of readeraw...,0,"[8294822, 10418456, 532313, 556431, 6317274, 2..."
14472576,14472576,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",How do we build a semantic parser in a new dom...,AbstractHow do we build a semantic parser in a...,0,"[2265838, 14341841, 6401679, 6216733, 2408319,..."
14323173,14323173,Speculation and negation annotation in natural...,[Veronika Vincze],0,0,0,"[5261517, 8410430, 18343028, 16285026, 4068646..."
442560,442560,HYENA-live: Fine-Grained Online Entity Type Cl...,"[Mohamed Yosef, Sandro Bauer, Johannes Hoffart...",0,AbstractRecent research has shown progress in ...,0,"[10977241, 8722811, 9345159, 6430811, 5570507,..."
44112954,44112954,UNAM at SemEval-2018 Task 10: Unsupervised Sem...,"[Ignacio Arroyo-Fernández, Iván Meza, Carlos-F...",0,AbstractIn this paper we report an unsupervise...,0,"[1919756, 17389337, 16694779, 63259515, 149407..."


In [221]:
p_ids_dict = {ids:1 for ids in p_ids}

In [222]:
df_target.ref_order.values[893]

['2575762',
 '2428290',
 '4248450',
 '2612947',
 '44063762',
 '51979529',
 '17703143',
 '908539',
 '49210174',
 '20587930',
 '40100965',
 '3626819',
 '52010710']

In [223]:
def del_targets_not_in_OUT_papers(ref_order):
    new_ref_order = []
    for num,ref in enumerate(ref_order):
        if not ref in p_ids_dict:
            print(num,ref)
        else:
            new_ref_order.append(ref)
    return new_ref_order

In [224]:
df_target['ref_order'] = df_target['ref_order'].apply(del_targets_not_in_OUT_papers)

7 908539
7 57352402
6 908539
4 908539
14 908539
8 908539
11 908539


In [225]:
for ref_order in df_target['ref_order'].values:
    for ref in ref_order:
        if not ref in p_ids_dict:
            print(num,ref)

In [226]:
df_target.head()

,acl_p_id,title,authors,abstract,abstract_grobid,abstract_latex,ref_order
10164018,10164018,Reader-Aware Multi-Document Summarization: An ...,"[Piji Li, Lidong Bing, Wai Lam]",We investigate the problem of reader-aware mul...,AbstractWe investigate the problem of readeraw...,0,"[8294822, 10418456, 532313, 556431, 6317274, 2..."
14472576,14472576,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",How do we build a semantic parser in a new dom...,AbstractHow do we build a semantic parser in a...,0,"[2265838, 14341841, 6401679, 6216733, 2408319,..."
14323173,14323173,Speculation and negation annotation in natural...,[Veronika Vincze],0,0,0,"[5261517, 8410430, 18343028, 16285026, 4068646..."
442560,442560,HYENA-live: Fine-Grained Online Entity Type Cl...,"[Mohamed Yosef, Sandro Bauer, Johannes Hoffart...",0,AbstractRecent research has shown progress in ...,0,"[10977241, 8722811, 9345159, 6430811, 5570507,..."
44112954,44112954,UNAM at SemEval-2018 Task 10: Unsupervised Sem...,"[Ignacio Arroyo-Fernández, Iván Meza, Carlos-F...",0,AbstractIn this paper we report an unsupervise...,0,"[1919756, 17389337, 16694779, 63259515, 149407..."


In [318]:
df_acl_features.head()

,paper_id,s2_pdf_hash,year,arxiv_id,acl_id,pmc_id,pubmed_id,doi,venue,abstract,title,authors,abstract_grobid,abstract_latex,cnt_out_grobid,cnt_out_latex,venue_cit_n,in_cits
13756507,13756507,3944c1740bb643415583363a0a35f12f8bd19a16,2018,1,0,0,0,1,arxiv,Recommendation systems play a vital role to ke...,Collaborative Memory Network for Recommendatio...,"[Travis Ebesu, Bin Shen, Yi Fang]",ABSTRACTRecommendation systems play a vital ro...,0,40,41,3450,13
14472576,14472576,f3de408be7d2e2720a61451bd196ac7e1ed9363a,2015,0,1,0,0,1,acl,How do we build a semantic parser in a new dom...,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",AbstractHow do we build a semantic parser in a...,0,27,0,2257,92
21369026,21369026,abd6baaf7e9d830feb3889e1019a989b6959ae89,2017,0,0,0,0,1,other,0,On the Uses of Word Sense Change for Research ...,"[Nina Tahmasebi, Thomas Risse]",0,0,29,0,0,0
2497200,2497200,5e3cd43b9c0288a08c782b8df1311dd3e4f06c9b,1994,0,0,0,0,1,TOIS,We describe an approach to text classification...,Information extraction as a basis for high-pre...,"[Ellen Riloff, Wendy Lehnert]",0,0,0,0,30,0
406684,406684,a68dc3c376272c5c08df211c6974f02380295ebd,2005,0,0,0,0,1,IEEE,"In many practical data mining applications, su...",Tri-training: exploiting unlabeled data using ...,"[Zhi-Hua Zhou, Ming Li]",0,0,30,0,1144,0


In [319]:
df_target.shape,df_acl_features.shape

((7259, 8), (26887, 18))

In [229]:
# df_target.to_excel(dir_name+'df_target.xlsx')
# df_target.to_csv(dir_name+'df_target.csv',encoding='utf8')
# df_acl_features.to_excel(dir_name+'df_acl_features.xlsx')
# df_acl_features.to_csv(dir_name+'df_acl_features.csv',encoding='utf8')

#### df_target удаляем дублирования в таргет и записываем как признак

In [320]:
df_target.iloc[0]['ref_order']

['8294822',
 '10418456',
 '532313',
 '556431',
 '6317274',
 '29562039',
 '13723748',
 '704517',
 '8377315',
 '16895865']

In [321]:
ref_order_uniq = []
doubled_values = []
for ref in df_target.iloc[0]['ref_order']:
    if ref in ref_order_uniq:
        doubled_values.append(ref)
    else:
        ref_order_uniq.append(ref)

In [322]:
def doubled_values(x):
    ref_order_uniq = []
    doubled_value = []
    for ref in x:
        if ref in ref_order_uniq:
            doubled_value.append(ref)
        else:
            ref_order_uniq.append(ref)
    return doubled_value

In [323]:
def remove_doubled_values(x):
    ref_order_uniq = []
    doubled_value = []
    for ref in x:
        if ref in ref_order_uniq:
            doubled_value.append(ref)
        else:
            ref_order_uniq.append(ref)
    return ref_order_uniq

In [324]:
df_target['doubl_ids'] = df_target.ref_order.apply(doubled_values)

In [325]:
df_target['ref_order'] = df_target.ref_order.apply(remove_doubled_values)

In [326]:
df_target.head()

,acl_p_id,title,authors,abstract,abstract_grobid,abstract_latex,ref_order,doubl_ids
10164018,10164018,Reader-Aware Multi-Document Summarization: An ...,"[Piji Li, Lidong Bing, Wai Lam]",We investigate the problem of reader-aware mul...,AbstractWe investigate the problem of readeraw...,0,"[8294822, 10418456, 532313, 556431, 6317274, 2...",[]
14472576,14472576,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",How do we build a semantic parser in a new dom...,AbstractHow do we build a semantic parser in a...,0,"[2265838, 14341841, 6401679, 6216733, 2408319,...",[]
14323173,14323173,Speculation and negation annotation in natural...,[Veronika Vincze],0,0,0,"[5261517, 8410430, 18343028, 16285026, 4068646...",[]
442560,442560,HYENA-live: Fine-Grained Online Entity Type Cl...,"[Mohamed Yosef, Sandro Bauer, Johannes Hoffart...",0,AbstractRecent research has shown progress in ...,0,"[10977241, 8722811, 9345159, 6430811, 5570507,...",[]
44112954,44112954,UNAM at SemEval-2018 Task 10: Unsupervised Sem...,"[Ignacio Arroyo-Fernández, Iván Meza, Carlos-F...",0,AbstractIn this paper we report an unsupervise...,0,"[1919756, 17389337, 16694779, 63259515, 149407...",[]


также удаляем дублирование в df_acl_features

In [327]:
df_acl_features.shape

(26887, 18)

In [328]:
df_acl_features.loc[~df_acl_features.index.duplicated(keep='first')].shape

(26886, 18)

In [329]:
df_acl_features = df_acl_features.loc[~df_acl_features.index.duplicated(keep='first')]

In [330]:
df_acl_features.shape

(26886, 18)

#### baseline model - Ranking by year

In [331]:
df_acl_features[df_acl_features.year != 0].year.astype(int).mean()

2009.4243285221805

In [332]:
df_acl_features[df_acl_features.year != 0].year.astype(int).median()

2011.0

In [333]:
def ranking_by_year(x,df_acl_features=df_acl_features,reverse=False):
    year_list_tups = []
    for out_paper_id in x:
        year = int(df_acl_features.loc[out_paper_id]['year']) 
        if year == 0:
            year_list_tups.append( (2011,out_paper_id) )
        else:
            year_list_tups.append( (year,out_paper_id) )
    year_list_tups = sorted(year_list_tups, key=lambda tup: tup[0],reverse=reverse)
    year_list_tups = list(map(lambda x: x[1],year_list_tups))
    return year_list_tups

In [334]:
## x = df_target.iloc[0]['ref_order']
x,len(x)

(['14680675',
  '14775471',
  '10253749',
  '15029034',
  '10608783',
  '9481819',
  '3201232',
  '15763200',
  '11125179',
  '2436152',
  '14476855',
  '5570507',
  '6272433',
  '9345159',
  '442560',
  '18169483',
  '8789606',
  '11174540',
  '4357791',
  '8760686',
  '7461625',
  '16915526',
  '7124381'],
 23)

In [335]:
year_list_tups = []
for out_paper_id in x:
    year_list_tups.append( (int(df_acl_features.loc[out_paper_id]['year']),out_paper_id) )
print(year_list_tups)
year_list_tups = sorted(year_list_tups, key=lambda tup: tup[0],reverse=False)
print(year_list_tups)
year_list_tups = list(map(lambda x: x[1],year_list_tups))
print(year_list_tups)

[(2006, '14680675'), (2012, '14775471'), (2010, '10253749'), (2013, '15029034'), (2013, '10608783'), (2014, '9481819'), (2018, '3201232'), (1992, '15763200'), (2014, '11125179'), (2012, '2436152'), (2013, '14476855'), (2002, '5570507'), (2010, '6272433'), (2012, '9345159'), (2013, '442560'), (2012, '18169483'), (2008, '8789606'), (2010, '11174540'), (2009, '4357791'), (2010, '8760686'), (2012, '7461625'), (2014, '16915526'), (2014, '7124381')]
[(1992, '15763200'), (2002, '5570507'), (2006, '14680675'), (2008, '8789606'), (2009, '4357791'), (2010, '10253749'), (2010, '6272433'), (2010, '11174540'), (2010, '8760686'), (2012, '14775471'), (2012, '2436152'), (2012, '9345159'), (2012, '18169483'), (2012, '7461625'), (2013, '15029034'), (2013, '10608783'), (2013, '14476855'), (2013, '442560'), (2014, '9481819'), (2014, '11125179'), (2014, '16915526'), (2014, '7124381'), (2018, '3201232')]
['15763200', '5570507', '14680675', '8789606', '4357791', '10253749', '6272433', '11174540', '8760686', 

In [336]:
df_target['rank_by_year'] = df_target.ref_order.apply(ranking_by_year)

In [343]:
(df_target.ref_order.apply(len) == df_target.rank_by_year.apply(len)).all()

True

In [344]:
df_target.head()

,acl_p_id,title,authors,abstract,abstract_grobid,abstract_latex,ref_order,doubl_ids,rank_by_year
10164018,10164018,Reader-Aware Multi-Document Summarization: An ...,"[Piji Li, Lidong Bing, Wai Lam]",We investigate the problem of reader-aware mul...,AbstractWe investigate the problem of readeraw...,0,"[8294822, 10418456, 532313, 556431, 6317274, 2...",[],"[8294822, 10418456, 532313, 13723748, 704517, ..."
14472576,14472576,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",How do we build a semantic parser in a new dom...,AbstractHow do we build a semantic parser in a...,0,"[2265838, 14341841, 6401679, 6216733, 2408319,...",[],"[62727207, 2498523, 9337134, 5667590, 10228634..."
14323173,14323173,Speculation and negation annotation in natural...,[Veronika Vincze],0,0,0,"[5261517, 8410430, 18343028, 16285026, 4068646...",[],"[8410430, 18343028, 5261517, 16285026, 4068646..."
442560,442560,HYENA-live: Fine-Grained Online Entity Type Cl...,"[Mohamed Yosef, Sandro Bauer, Johannes Hoffart...",0,AbstractRecent research has shown progress in ...,0,"[10977241, 8722811, 9345159, 6430811, 5570507,...",[],"[5570507, 10977241, 33008026, 8722811, 6430811..."
44112954,44112954,UNAM at SemEval-2018 Task 10: Unsupervised Sem...,"[Ignacio Arroyo-Fernández, Iván Meza, Carlos-F...",0,AbstractIn this paper we report an unsupervise...,0,"[1919756, 17389337, 16694779, 63259515, 149407...",[],"[6708547, 10677711, 14940757, 10301835, 141718..."


Посчитаем ранговый коэффициент корреляции Кендалла

In [345]:
from scipy import stats

In [353]:
x1,x2 = np.array(df_target.iloc[0]['ref_order']).astype(int),np.array(df_target.iloc[0]['rank_by_year']).astype(int)

In [354]:
tau, _ = stats.kendalltau(x1, x2)
# Kendall’s tau is a measure of the correspondence between two rankings. 
# Values close to 1 indicate strong agreement, values close to -1 indicate strong disagreement

In [355]:
tau

0.06666666666666667

In [358]:
def measure_kendalltau_betw_2_str_lists(x):
    ref_order = np.array(x['ref_order']).astype(int)
    rank_by_year = np.array(x['rank_by_year']).astype(int)
    tau, _ = stats.kendalltau(ref_order, rank_by_year)
    return tau

In [359]:
measure_kendalltau_betw_2_str_lists(df_target.iloc[0])

0.06666666666666667

In [360]:
df_target['baseline_kendalltau'] = df_target.apply( measure_kendalltau_betw_2_str_lists, axis=1)

In [361]:
df_target.head()

,acl_p_id,title,authors,abstract,abstract_grobid,abstract_latex,ref_order,doubl_ids,rank_by_year,baseline_kendalltau
10164018,10164018,Reader-Aware Multi-Document Summarization: An ...,"[Piji Li, Lidong Bing, Wai Lam]",We investigate the problem of reader-aware mul...,AbstractWe investigate the problem of readeraw...,0,"[8294822, 10418456, 532313, 556431, 6317274, 2...",[],"[8294822, 10418456, 532313, 13723748, 704517, ...",0.066667
14472576,14472576,Building a Semantic Parser Overnight,"[Yushi Wang, Jonathan Berant, Percy Liang]",How do we build a semantic parser in a new dom...,AbstractHow do we build a semantic parser in a...,0,"[2265838, 14341841, 6401679, 6216733, 2408319,...",[],"[62727207, 2498523, 9337134, 5667590, 10228634...",-0.098039
14323173,14323173,Speculation and negation annotation in natural...,[Veronika Vincze],0,0,0,"[5261517, 8410430, 18343028, 16285026, 4068646...",[],"[8410430, 18343028, 5261517, 16285026, 4068646...",0.466667
442560,442560,HYENA-live: Fine-Grained Online Entity Type Cl...,"[Mohamed Yosef, Sandro Bauer, Johannes Hoffart...",0,AbstractRecent research has shown progress in ...,0,"[10977241, 8722811, 9345159, 6430811, 5570507,...",[],"[5570507, 10977241, 33008026, 8722811, 6430811...",0.047619
44112954,44112954,UNAM at SemEval-2018 Task 10: Unsupervised Sem...,"[Ignacio Arroyo-Fernández, Iván Meza, Carlos-F...",0,AbstractIn this paper we report an unsupervise...,0,"[1919756, 17389337, 16694779, 63259515, 149407...",[],"[6708547, 10677711, 14940757, 10301835, 141718...",-0.010989


In [362]:
np.mean(df_target.baseline_kendalltau),np.median(df_target.baseline_kendalltau)

(0.10070598365623208, 0.06666666666666667)